In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from PIL import Image

# Create content folder if needed
os.makedirs('content', exist_ok=True)

# Step 1: Load and encode your known face
def load_and_encode_known_face(image_path):
    """Load your known photo and extract face encoding"""
    # Load image
    known_image = face_recognition.load_image_file(image_path)
    
    # Get face encodings
    known_encodings = face_recognition.face_encodings(known_image)
    
    if len(known_encodings) > 0:
        print("✅ Known face encoded successfully!")
        return known_encodings[0]
    else:
        print("❌ No face found in the known image")
        return None

# Load your known face
known_face_encoding = load_and_encode_known_face('content/Sample_CV.webp')

if known_face_encoding is None:
    print("Please make sure your photo has a clear face")
    exit()

# Step 2: Real-time recognition
def recognize_faces():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    print("Starting face recognition... Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Convert BGR to RGB (face_recognition uses RGB)
        rgb_frame = frame[:, :, ::-1]
        
        # Find all faces in current frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Compare with known face
            matches = face_recognition.compare_faces([known_face_encoding], face_encoding)
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)
            
            # Get face coordinates
            top, right, bottom, left = face_location
            
            if matches[0]:
                # It's a match! (You)
                color = (0, 255, 0)  # Green
                label = "YOU! ({:.2f})".format(distance[0])
            else:
                # Not a match (Someone else)
                color = (0, 0, 255)  # Red
                label = "Unknown ({:.2f})".format(distance[0])
            
            # Draw rectangle and label
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.putText(frame, label, (left, top-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Display info
        cv2.putText(frame, f'Faces: {len(face_locations)}', (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Show frame
        cv2.imshow('Face Recognition - Is it YOU?', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Start recognition
recognize_faces()